# Using NannyML Concept Shift Algorithm from AWS Marketplace 

## Overview

Machine Learning Models are able to learn a relationship between some variables, **X** and a target Y.
The relationship - otherwise called concept - learnt from the training data is static.
However the world in which our model operates is not static.
We expand in the possible types of change in our blog [Understanding Data Shift](https://www.nannyml.com/blog/types-of-data-shift).
Here we focus on Concept Shift and present an algorithm that captures its impact.

## Algorithm Overview

How do we measure the impact of Concept Shift? This is a complex problem, our data rarely, if ever contains just concept shift for us to measure it in a straightforward way. In order to measure only the effects of Concept Shift our algorithm answers the following question:

**What would the performance of my model be on the reference dataset, if the world works as described by the current data.**

In the place of current data we use the data contained in the [data chunk](https://nannyml.readthedocs.io/en/stable/glossary.html#term-Data-Chunk)
for which we are estimaing the impact of Concept Shift.

The algorithm consists of the following steps:

1. Learn the latest concept from model feature columns and targets on chunk data.
2. Make predictions on reference data using learnt concept model.
3. Estimate the performance of the client model assuming learnt concept predictions are ground truth.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [Model Concept Shift Detection - NannyML](https://aws.amazon.com/marketplace/pp/prodview-64nptz3lrs4gc). 

## Contents
1. [Subscribe to the algorithm](#1.-Subscribe-to-the-algorithm)
1. [Prepare dataset](#2.-Prepare-dataset)
	1. [Dataset format expected by the algorithm](#A.-Dataset-format-expected-by-the-algorithm)
	1. [Visualize reference dataset](#B.-Visualize-reference-dataset)
	1. [Upload datasets to Amazon S3](#C.-Upload-datasets-to-Amazon-S3)
1. [Train the machine learning algorithm](#3:-Train-the-machine-learning-algorithm)
	1. [Set up environment](#3.1-Set-up-environment)
	1. [Train a model](#3.2-Train-the-algorithm)
1. [Deploy model](#4:-Deploy-model)
1. [Perform Batch Inference](#5.-Perform-Batch-Inference)
1. [Clean-up](#6.-Clean-up)
	1. [Delete the model](#A.-Delete-the-model)
	1. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))


## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the algorithm

To subscribe to the algorithm:
1. Open the algorithm listing page [Model Concept Shift Detection - NannyML](https://aws.amazon.com/marketplace/pp/prodview-64nptz3lrs4gc)
1. On the AWS Marketplace listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

<font color='red'>Directly copy your assigned ARN code below:<font>

In [1]:
algo_arn = "<Customer to specify algorithm ARN corresponding to their AWS region>"

## 2. Prepare dataset

In [2]:
import sagemaker as sage
from sagemaker import get_execution_role
import pandas as pd
import json

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


### A. Dataset format expected by the algorithm

Currently the Concept Shift detection algorithm only works for Binary Classification problems. 
Hence for this notebook we will be using 
[NannyML's synthetic car loan dataset](https://nannyml.readthedocs.io/en/stable/datasets/binary_car_loan.html).

More detailed information on what data are required can be found on [NannyML's Data Requirements Documentation](https://nannyml.readthedocs.io/en/stable/tutorials/data_requirements.html).

For AWS Algorithm marketplace it's best if data are provided in a CSV format. You can find some information about dataset format in **Usage Information** section of [Model Concept Shift Detection - NannyML](https://aws.amazon.com/marketplace/pp/prodview-64nptz3lrs4gc).
<br>

<font color='red'>Edit code below as appropriate for the Machine Learning problem type you are interested in:<font>

### B. Visualize reference dataset

In [3]:
reference_dataset = "data/synthetic_car_loan_reference.csv"
pd.read_csv(reference_dataset).head()

,car_value,salary_range,debt_to_income_ratio,loan_length,repaid_loan_on_prev_car,size_of_downpayment,driver_tenure,repaid,timestamp,y_pred_proba,y_pred
0,39811.0,40K - 60K €,0.632950,19.0,False,40%,0.212653,1.0,2018-01-01 00:00:00.000,0.99,1
1,12679.0,40K - 60K €,0.718627,7.0,True,10%,4.927549,0.0,2018-01-01 00:08:43.152,0.07,0
2,19847.0,40K - 60K €,0.721724,17.0,False,0%,0.520817,1.0,2018-01-01 00:17:26.304,1.00,1
3,22652.0,20K - 20K €,0.705992,16.0,False,10%,0.453649,1.0,2018-01-01 00:26:09.456,0.98,1
4,21268.0,60K+ €,0.671888,21.0,True,30%,5.695263,1.0,2018-01-01 00:34:52.608,0.99,1


### C. Upload datasets to Amazon S3

In [4]:
sagemaker_session = sage.Session()
bucket = sagemaker_session.default_bucket()
# Uncomment below to see selected default bucket
# bucket

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [5]:
demo_prefix = "doc-notebook-demo"

In [6]:
reference_data = sagemaker_session.upload_data(
    reference_dataset, bucket=bucket, key_prefix=demo_prefix
)

## 3: Train the machine learning algorithm

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to train a machine learning model. 

### 3.1 Set up environment

In [7]:
role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [8]:
output_location = f"s3://{bucket}/{demo_prefix}/output"
# Uncomment below to see specified output location for Algorithm artifacts and outputs
# output_location

### 3.2 Train the algorithm

In order to train the algorithm there are two groups of parameters that need to be specified, AWS Sagemaker parameters and NannyML Concept Shift detection parameters. You can also find more information about dataset format and AWS Sagemaker parameters in **Hyperparameters** section of [Model Concept Shift Detection - NannyML](https://aws.amazon.com/marketplace/pp/prodview-64nptz3lrs4gc).

The NannyML Concept Shift detection parameters needing to be specified are:

- **feature_column_names:** A list with the names of the columns used as inputs from the model that
    is evaluated.
- **y_pred_proba:** The name of the column in the reference data that
contains the predicted probabilities.
- **y_pred:** The name of the column in the reference data that
contains the predicted classes.
- **y_true:** The name of the column in the reference data that
contains the true classes.
- **timestamp_column_name (Optional):** The name of the column in the reference data that
contains timestamps.
- **metrics:** A list of metrics for which to estimate impact of concept shift. Available options are
`accuracy`, `precision`, `recall`, `specificity`, `f1`, `roc_auc` and `magnitude`. Magnitude is a custom concept shift measure ranging between 0, for no concept shift, to 1, theoritical maximum concept shift.
- **chunk_size (Optional):** The number of observations in each chunk of data
used. Only one chunking argument needs to be provided. For more information about
[chunking](https://nannyml.readthedocs.io/en/stable/glossary.html#term-Data-Chunk)
configurations check out the [chunking tutorial](https://nannyml.readthedocs.io/en/stable/tutorials/chunking.html#chunking).
- **chunk_number (Optional):** The number of chunks to be created out of data provided for each
[period](https://nannyml.readthedocs.io/en/stable/tutorials/data_requirements.html#data-periods).
- **chunk_period (Optional):** The time period based on which we aggregate the provided data in
order to create chunks.
- **chunker (Optional):** A NannyML [Chunker object](https://nannyml.readthedocs.io/en/stable/nannyml/nannyml.chunk.html#nannyml.chunk.Chunker) that will handle the aggregation
provided data in order to create chunks.
- **problem_type:** A string indicating the type of problem being monitored. Currently only `"classification_binary"` problems are supported for Concept Shift.
- **thresholds (Optional):** The thresholds used to calculate the alert flag. For more information about
thresholds, check out the [thresholds tutorial](https://nannyml.readthedocs.io/en/stable/tutorials/thresholds.html#thresholds).
- **hyperparameters (Optional):** A dictionary containing hyperparameter specification for the training of the
*LGBMClassifier* concept model.
- **tune_hyperparameters:** A boolearn specifying whether hyperparameter tuning should be
performed. False by default.
- **hyperparameter_tuning_config (Optional):** A dictionary containing hyperparameter tuning specification for the hyperparameter tuning of the *LGBMClassifier* concept model.


The AWS Sagemaker parameters needing to be specified are:

- **problem_type:** A string indicating the type of problem being monitored. Currently only`"classification_binary"` problems are supported for Concept Shift.
- **data_filename:** A string with the file name that contains the training data.
- **data_type:** The file format of the training data file. `csv` is the recomended option.
- **parameters:** Algorithm parameters dict, encoded as JSON string. This parameters are passed as kwargs to the corresponding algorithm depending the problem type.

<font color='red'>Edit code below as appropriate for your use case. Note that you need to know the list of model `feature_column_names`:<font>

In [9]:
# Define hyperparameters
nannyml_parameters = {
    "y_pred_proba": "y_pred_proba",
    "y_pred": "y_pred",
    "y_true": "repaid",
    "timestamp_column_name": "timestamp",
    "metrics": ["roc_auc", 'magnitude'],
    "chunk_size": 5000,
    "problem_type": "classification_binary",
    "feature_column_names": [
        'car_value', 'salary_range', 'debt_to_income_ratio', 'loan_length', 'repaid_loan_on_prev_car', 'size_of_downpayment', 'driver_tenure'
    ],
}
# json.dumps needed due to sagemaker specifications
sagemaker_hyperparameters = {
    "data_filename": reference_dataset.split("/")[-1],
    "data_type": "csv",
    "problem_type": "classification_binary",
    "parameters": json.dumps(nannyml_parameters),
}

For information on creating an `Estimator` object, see [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

In [10]:
# Create an estimator object for running a training job
estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name='nml-concept-shift',
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    hyperparameters=sagemaker_hyperparameters,
)

In [11]:
# Run the training job.
estimator.fit(
    {'training': reference_data}
)

INFO:sagemaker:Creating training-job with name: nml-concept-shift-2023-11-01-18-03-15-619


2023-11-01 18:03:15 Starting - Starting the training job...
2023-11-01 18:03:32 Starting - Preparing the instances for training......
2023-11-01 18:04:26 Downloading - Downloading input data...
2023-11-01 18:05:10 Training - Downloading the training image........INFO:nannyml:Logger object created.
INFO:nannyml:Hyperparameters read.
INFO:nannyml:RCS Estimator Instantiated.
INFO:nannyml:Data loaded.
INFO:nannyml.base:fitting nannyml_premium.concept_shift.rcs.calculator.ClassificationConceptShiftEstimator
DEBUG:nannyml.usage_logging:found NML_DISABLE_USAGE_LOGGING key in environment variables. Usage event UsageEvent.RCS_ESTIMATOR_FIT not logged.
INFO:nannyml:Estimator fit.
INFO:nannyml.io.store.base:storing object "nannyml_premium.concept_shift.rcs.calculator.ClassificationConceptShiftEstimator" to store "nannyml.io.store.file_store.FilesystemStore"
DEBUG:fsspec.local:open file: /opt/ml/model/estimator.pkl
DEBUG:nannyml.io.store.serializers:serializing object nannyml_premium.concept_shift

See this [blog-post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/) for more information how to visualize metrics during the process. You can also open the training job from [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/home?#/jobs/) and monitor the metrics/logs in **Monitor** section.

## 4: Deploy model

**NannyML's Concept Shift Detection is not designed for real time inference, therefore it is not recommended to use it in this way.**

For this reason we are not showcasing the real-time inference feature of Sagemaker Algorithm.

## 5. Perform Batch Inference

In this section, you will perform batch inference using multiple input payloads together.

In [12]:
# upload the batch-transform job input files to S3
inference_dataset = "data/synthetic_car_loan_analysis_with_targets.csv"
inference_data = sagemaker_session.upload_data(inference_dataset, bucket=bucket, key_prefix=demo_prefix)
# print("Transform input uploaded to " + inference_data)

In [13]:
# Run the batch-transform job
transformer = estimator.transformer(
    instance_count=1,
    instance_type="ml.m5.large",
    output_path=output_location
)
transformer.transform(inference_data, content_type="text/csv")
transformer.wait()

INFO:sagemaker:Creating model package with name: nannyml-reverse-concept-shift--84a2df2a-2023-11-01-18-07-29-223


.........

INFO:sagemaker:Creating model with name: nannyml-reverse-concept-shift--84a2df2a-2023-11-01-18-08-14-710


INFO:sagemaker:Creating transform job with name: nml-concept-shift-2023-11-01-18-08-15-191


............................... * Serving Flask app 'inference_server'
 * Debug mode: off
#033#033WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.#033
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://127.0.0.1:8080
#033Press CTRL+C to quit#033
169.254.255.130 - - [01/Nov/2023 18:13:31] "GET /ping HTTP/1.1" 200 -
169.254.255.130 - - [01/Nov/2023 18:13:31] "#033GET /execution-parameters HTTP/1.1#033" 404 -
Received POST invocation request.
found request data
created string io object
set buffer to 0
Created pandas object.
Estimation invoked with 50000 rows
Estimation invoked with columns: ['car_value', 'salary_range', 'debt_to_income_ratio', 'loan_length', 'repaid_loan_on_prev_car', 'size_of_downpayment', 'driver_tenure', 'timestamp', 'y_pred_proba', 'y_pred', 'repaid']
found request data
created string io object
set buffer to 0
Created pandas object.
Estimation invoked with

**View Results of Performance Estimation**

In [14]:
results = pd.read_csv(transformer.output_path + "/" + inference_dataset.split("/")[-1] + ".out", header = [0,1])
# results.to_csv("data/synthetic_car_loan_rcs_output.csv", index=False)
results

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


chunk                                                         \
              key chunk_index start_index end_index           start_date   
0        [0:4999]           0           0      4999  2018-01-01 00:00:00   
1     [5000:9999]           1        5000      9999  2018-01-31 06:36:00   
2   [10000:14999]           2       10000     14999  2018-03-02 13:12:00   
3   [15000:19999]           3       15000     19999  2018-04-01 19:48:00   
4   [20000:24999]           4       20000     24999  2018-05-02 02:24:00   
5   [25000:29999]           5       25000     29999  2018-06-01 09:00:00   
6   [30000:34999]           6       30000     34999  2018-07-01 15:36:00   
7   [35000:39999]           7       35000     39999  2018-07-31 22:12:00   
8   [40000:44999]           8       40000     44999  2018-08-31 04:48:00   
9   [45000:49999]           9       45000     49999  2018-09-30 11:24:00   
10       [0:4999]           0           0      4999  2018-10-30 18:00:00   
11    [5000:9999]           1        5000      9999  2018-11-30 00:36:00   
12  [10000:14999]           2       10000     14999  2018-12-30 07:12:00   
13  [15000:19999]           3       15000     19999  2019-01-29 13:48:00   
14  [20000:24999]           4       20000     24999  2019-02-28 20:24:00   
15  [25000:29999]           5       25000     29999  2019-03-31 03:00:00   
16  [30000:34999]           6       30000     34999  2019-04-30 09:36:00   
17  [35000:39999]           7       35000     39999  2019-05-30 16:12:00   
18  [40000:44999]           8       40000     44999  2019-06-29 22:48:00   
19  [45000:49999]           9       45000     49999  2019-07-30 05:24:00   

                                         roc_auc                 \
                   end_date     period     value sampling_error   
0   2018-01-31 06:27:16.848  reference  0.970999       0.001811   
1   2018-03-02 13:03:16.848  reference  0.964029       0.001811   
2   2018-04-01 19:39:16.848  reference  0.968708       0.001811   
3   2018-05-02 02:15:16.848  reference  0.963728       0.001811   
4   2018-06-01 08:51:16.848  reference  0.960701       0.001811   
5   2018-07-01 15:27:16.848  reference  0.962452       0.001811   
6   2018-07-31 22:03:16.848  reference  0.966094       0.001811   
7   2018-08-31 04:39:16.848  reference  0.964554       0.001811   
8   2018-09-30 11:15:16.848  reference  0.968280       0.001811   
9   2018-10-30 17:51:16.848  reference  0.962080       0.001811   
10  2018-11-30 00:27:16.848   analysis  0.962994       0.001811   
11  2018-12-30 07:03:16.848   analysis  0.965751       0.001811   
12  2019-01-29 13:39:16.848   analysis  0.969490       0.001811   
13  2019-02-28 20:15:16.848   analysis  0.961620       0.001811   
14  2019-03-31 02:51:16.848   analysis  0.963987       0.001811   
15  2019-04-30 09:27:16.848   analysis  0.956955       0.001811   
16  2019-05-30 16:03:16.848   analysis  0.933329       0.001811   
17  2019-06-29 22:39:16.848   analysis  0.942431       0.001811   
18  2019-07-30 05:15:16.848   analysis  0.952643       0.001811   
19  2019-08-29 11:51:16.848   analysis  0.935890       0.001811   

                              ...                                         \
   upper_confidence_boundary  ... upper_threshold lower_threshold  alert   
0                   0.976431  ...        0.974539        0.955786  False   
1                   0.969461  ...        0.974539        0.955786  False   
2                   0.974140  ...        0.974539        0.955786  False   
3                   0.969160  ...        0.974539        0.955786  False   
4                   0.966133  ...        0.974539        0.955786  False   
5                   0.967884  ...        0.974539        0.955786  False   
6                   0.971526  ...        0.974539        0.955786  False   
7                   0.969986  ...        0.974539        0.955786  False   
8                   0.973712  ...        0.974539        0.955786  False   
9                   0.967512

## 6. Clean-up

### A. Delete the model

In [15]:
transformer.delete_model()

INFO:sagemaker:Deleting model with name: nannyml-reverse-concept-shift--84a2df2a-2023-11-01-18-08-14-710


### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

